# [3-22] Exploratory Data Analysis
---
<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Data_visualization_process_v1.png" style="width: 500px; height: 400px;" />


This lab covers the Tukey approach to exploratory data analysis, bringing together key concepts and skills from earlier in the course like data visualization and the train-test-validate split. We will also introduce dimensionality reduction in the form of Principal Components Analysis.

*Estimated Time: 45 minutes*

---

### Table of Contents


1 - [Intro to Exploratory Data Analysis](#section 1)<br>

2 - [EDA and Visualization](#section 2)<br>

3 - [Dimensionality Reduction](#section 3)<br>




**Dependencies:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

!pip install nltk
import nltk
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

!pip install plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

---
## 1. Introduction to Exploratory Data Analysis <a id='section 1'></a>

> "It is important to understand what you CAN DO before you learn to measure how WELL you seem to have done it." -John W. Tukey, *Exploratory Data Analysis*

Previous labs and many introductory data analysis classes like Data-8 focus on **Confirmatory Data Analysis (CDA)**: the use of statistical methods to see whether the data supports a given hypothesis or model. But, where does the hypothesis come from in the first place?

**Exploratory Data Analysis (EDA)** is the process of 'looking at data to see what it seems to say'. Through this process, we hope to accomplish several things:
- learn about the overall 'shape' of the data: structure, organization, ranges of values
- assess what assumptions we can make about the data as a basis for later statistical inference
- figure out the appropriate tools and techniques for analysis 
- tentatively create testable, appropriate hypotheses or models

EDA is complementary to CDA- in fact, CDA is often only possible after having done EDA. This is clear when compared to the methods of natural sciences: a scientist's research starts with making observations about the world, which leads to questions about what they see. Only after observing and wondering can we begin to systematically test our hypotheses. Similarly, every data analysis project you do should begin with EDA.

Today, we'll go through the EDA process for speech transcripts from [2016 US Presidential campaigns](http://www.presidency.ucsb.edu/2016_election.php). You'll use many coding skills from previous labs. And because EDA is an exploratory process, there are many ways to approach the code (we'll give some tips and guidance). 

In [ ]:
# load the data
campaign = pd.read_csv("data/campaign_2016.csv", index_col=0)
campaign.head()

A handy way to get basic summary statistics about your data is using the `.describe()` method on your table. Other useful methods are:
- `.unique()` to look at a column's unique values
- `.value_counts()` to see how often a column's values occur.
- `.isnull()` to screen for null values

From these few methods, we can immediately see things that will affect future analysis options.


In [ ]:
# use pandas methods to broadly examine the scope and values of the data

...


**QUESTION**:
Name at least two issues seen using `describe`, `unique`, `value_counts`, and `isnull`. Why would each issue affect future analysis?
Hint for analyzing `describe`: Which statistics would we expect to be the same for all columns, and are they actually the same? Which column would we expect 'count' to be different from 'unique', and are they?

**ANSWER:**


As you can see, there are a handful of null values in the 'Text' column. Null values can mess up calculations and need to be dealt with by filling them with a dummy value or deleting the row. Because the null values are for text, and because we're most interested in the text content, we'll drop the offending rows. How you deal with null values will vary on a project-by-project basis.


In [ ]:
# drop the null-valued rows with dropna
...

### Feature extraction
Many of numpy and pandas' summary and aggregation methods are built for numerical data, not text. To better understand our data, we use **feature extraction** to create features from transformations of the input features. Some useful features for text would be:
- the number of characters per text
- the number of words per text
- the number of sentences per text
- the average length of words per text
- the average length of sentences per text

Since we're not focusing on data cleaning today (remember the 3/1 lab?), we've provided the code to add columns with lowercase text, word tokens, and sentence tokens that you might find useful. You may also want to revisit the pandas [str methods](https://pandas.pydata.org/pandas-docs/stable/text.html#method-summary).

Note: During EDA, it is always best practice to clearly document what transformations you did, and to add transformed data in new columns rather than transforming the data in place. This will save you a lot of trouble if you need to refer to the original data in the future!

In [ ]:
# columns with lowercase, word tokens, sentence tokens
campaign['lower_text'] = campaign['Text'].str.lower()
campaign['words'] = campaign['lower_text'].apply(nltk.word_tokenize)
campaign['sentences'] = campaign['lower_text'].apply(nltk.sent_tokenize)

campaign.head()

In [ ]:
# extract features from the text data
campaign['char_count'] = ...
campaign['word_count'] = ...
campaign['sentence_count'] = ...
campaign['avg_word_length'] = ...
campaign['avg_sentence_length'] = ...

The 'Date' column currently stores dates as strings, which we can't easily compare on a graph. Fortunately, pandas has a **datetime** object that can be easily ordered. Add a column containing string dates that have been transformed into datetime objects. [There's a function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) that might be helpful here...

In [ ]:
# convert the 'Date' column to type datetime
campaign['datetime'] = ...
campaign.head()

We also might be interested in seeing how candidates use particular words. Using a for loop, create a new column for each topic word containing the number of times the word is used in each text.

Hint: take a look at pandas str methods

In [ ]:
# create list of possible topics
topics = ['nuclear', 'peace', ' war',
          'north korea', 'russia', 'terror', 'fake news', 'email',
          ' poverty', ' human rights', ' abortion', ' refugee', ' immigration',
          'equality', ' democracy', ' freedom', 'vote', 'energy', 'oil',  'coal',  ' income',
          'economy', 'growth', 'inflation', 'climate change', 'security',
          'cyber', 'trade', 'inequality', 'pollution', 'global warming',
          'education', 'health', 'infrastructure', 'regulation', 'nutrition', 'transportation',
          'violence', 'agriculture', 'crime', 'drugs', 'obesity',
          'islam', 'housing', 'sustainable']


for i in topics:    
    campaign[i] = ...



Now that we have numerical data, try using `describe()` again. What's changed now that there's numerical data? What else can you say about this data set?

In [ ]:
...

## 2. EDA and Visualization <a id='section 2'></a>

> "The greatest value of a picture is when it *forces* us to notice what we never expected to see." - John W. Tukey, *Exploratory Data Analysis*

Our data set contains almost 7500 records, which is difficult for our brains to parse as a whole. And, this would be considered a fairly small data set by most data science standards. Visualizations are an important tool for summarizing and understanding a data set.

In EDA, we're often interested in a few characteristics:
- where the data is centered (the mean or median)
- how much the data varies from the mean (the spread/variance/standard deviation)
- outliers (the min and max)

The **box plot** is a useful way to visualize all of these things.
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Elements_of_a_boxplot_en.svg/640px-Elements_of_a_boxplot_en.svg.png" />
The rectangular box of the plot holds 50% of the data points. The bottom of the box represents the point that is higher in value than 25% of the data (the lower quartile), and the top of the box represents the point that is higher than 75% of the data (the upper quartile). The whiskers extend on each side to the lowest data point that is within 1.5 times the length of the box (the inter-quartile range, or IQR). All points farther out are graphed as circles (the outliers).

We can make box plots with `plt.boxplot`:

In [ ]:
# make a box plot of the word counts
plt.boxplot(campaign['word_count'])
plt.ylabel('number words')
plt.title('Word Counts');

That's an awfully scrunched-up box plot! We can see that over half of our texts contain less than a thousand words, but many texts contain several times that. 

It may be helpful to sort out the different types of texts. Create a dataframe that only has type 'speech', and another dataframe that only has type 'statement'. Then, make a box plot for each. Compare the box plots for word counts to those for word length, sentence length, or character count.

In [ ]:
# filter out everything except speeches
speeches = ...

# make a box plot
...

In [ ]:
# filter out everything but statements
statements = ...

# make a box plot
...

__QUESTION__: what can you conclude about the text data from these box plots?

**ANSWER**: 

Box plots pack a lot of information, but the plots can get muddy if many data points overlap. A **Violin plot** is similar to a box plot, but it also shows the distribution of the data at different values (you can think of it like a smoothed histogram).

Look at the word counts, word lengths, and other features using `plt.violinplot`.

In [ ]:
# make a violin plot
...

In [ ]:
# make another violin plot
...

**QUESTION**: What can you see with the violin plot that you couldn't with the box plot?

**SOLUTION**: 

Unexpected or interesting information may also be found by looking at multiple features together, or by looking at subgroups. 

If you need a refresher on grouping in pandas:

1. select the features you need using `.loc[<rows>, <columns>]`. The rows and columns can be a single label, a list of labels, or a range (e.g. `'word_count':` would give you all columns to the right of and including `word_count`).
2. call `.groupby` with the label of the feature you want to group by
3. call the function you want to use to aggregate the values in each group (e.g. `sum()`, `min()`, `max()`, `mean()`...)

For example, here's a bar plot of the average word count in each candidate's speeches using groupby:


In [ ]:
by_candidate = (speeches.loc[:, ['Candidate', 'inequality', 'russia', 'nuclear']] # select all rows, only the columns needed
            .groupby('Candidate') # make groups of word counts for each candidate
            .mean()) # take the average of each group of candidate's word counts

by_candidate.plot.bar(figsize=(12, 5)); # make a bar plot; figsize embiggens it


In the cells below, try creating:
- a line plot showing the average number of times the word 'email' was used on each date (use groupby on the `datetime` column, not `Date`)
- histograms showing the distribution of `word_length` for Democrats and Republicans
- a scatter plot plotting the length of words against the length of sentences for speeches and statements

Refer to the [pyplot](https://matplotlib.org/tutorials/introductory/pyplot.html#sphx-glr-tutorials-introductory-pyplot-py) and [pandas dataframe plot](https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-plotting) docs as needed. Feel free to experiment with other groups or features!

In [ ]:
...

In [ ]:
...

In [ ]:
...

**QUESTION**: What kinds of interesting/suspicious patterns or outliers did you see?
Given what you now know about the data set, what kinds of hypotheses could we test on the data?

**ANSWER**:

## 3. Principal Components Analysis <a id='section 3'></a>

We've seen that with text analysis, feature matrices get very large, very fast (remember the tf-idf matrix?). To help us work with high-dimensional data, we can use **Principal Component Analysis (PCA)**.

The goal of PCA is to look for patterns in the data (i.e. correlations between variables). If two variables are strongly correlated, including just one of them in the model might represent the data about as good as if we had them both. PCA looks for variables that account for as much of the variance in the data as possible. This makes it great for **dimensionality reduction**: getting rid of features (dimensions) that don't explain much of the variance in the data so we can more easily manipulate, visualize, and understand the data.

Note: the details of PCA are hard to follow without some knowledge of linear algebra. If you're interested, check out a more in-depth example [here](https://plot.ly/ipython-notebooks/principal-component-analysis/).

Let's explore reducing the dimensions for our text data. We'll start by using the features we extracted. First, create a table `X` that only has the columns with the extracted numerical features (no string or datetime values) and an array `y` that has the names of the candidates (this will help us visualize our data). 

In [ ]:
# select only the columns with int or float values
X = campaign.iloc[:, 9:].drop('datetime', axis=1)

# select the column with the candidate names
y = campaign['Candidate']

Next, we want to standardize X so that all the features have the same scale (i.e. have an average value of 0 and a standard deviation of 1). Do this by creating a `StandardScaler()`, then run its `fit_transform` method on `X`. You should recognize the syntax from other scikit-learn models.

In [ ]:
# scale the data
X_std = ...

# look at the covariance matrix
np.cov(X_std.T)

Finally, initialize a `PCA` object (set `n_components=2` so we can graph it in two dimensions) and use its `fit_transform` method on your standardized X to get `Y_pca`: the top n (in this case 2) principal components that explain variance in the data.

In [ ]:
# initialize PCA
...

# fit the standardized data
Y_pca = ...

We can see how much variance is being explained by these components by using `.explained_variance_ratio`. This gives the proportions of the total variance in documents that are explained by each component in descending order.

In [ ]:
# get the explained variance ratios
pca.explained_variance_ratio_

**QUESTION**: if our PCA was fit using ALL possible components (that is, n_components was set to the original number of features in the data set), what should the explained variance ratios add up to?

**ANSWER**: The ratios should add up to 1: including all the features means that all the variance can be explained.


To tell the candidates apart on our scatter plot, we'll assign each a different color: blues for Democrats, and reds for Republicans. Run the next cell to create the custom palette.

In [ ]:
# separate candidates by party and pair each candidate with a shade of their party color
dems = zip(campaign[campaign.Party=='D'].Candidate.unique(), sns.color_palette('Blues', 5).as_hex())
gops = zip(campaign[campaign.Party=='R'].Candidate.unique(), sns.color_palette('Reds', 16).as_hex())

# collect the candidate-color pairs in a dictionary
candidate_colors = {}

for dem, color in dems:
    candidate_colors[dem] = color

for gop, color in gops:
    candidate_colors[gop] = color

# create the ordered color palette
colors = candidate_colors.values()

# show the candidates and palette
print(candidate_colors.keys())
sns.palplot(colors)

Finally, plot the data by running the cell below.


In [ ]:
# create a dataframe with the principal components and the classes
components = pd.DataFrame({'component 1': Y_pca[:, 0],
                          'component 2': Y_pca[:, 1],
                          'Candidate': y})

colors = candidate_colors.values()

# plot
sns.lmplot(x = 'component 1', y = 'component 2', data = components, hue='Candidate', legend=True, fit_reg=False, 
            hue_order=candidate_colors.keys(),palette=colors, scatter_kws={'s':5}, size=7);

Using PCA on our derived features (e.g. word counts), we can see some separation between different people. However most of the data points are clustered in the same area, and there's a lot of overlap. Would we see something different if we had done dimensionality reduction on a different numerical representation of the data?

Let's revisit the tf-idf. As we saw Tuesday, a tf-idf is very high-dimensional, giving information about the frequency of a term in a document and across all documents for every possible term in every document. We can do dimensionality reduction on a tf-idf matrix, too, using **Latent Semantic Analysis (LSA)**. 

Like PCA, LSA uses linear algebra to reduce dimensionality while still representing the variation between documents. The code is very similar too, except in this case we don't have to standardize the feature data. First, create a `TfidfVectorizer()` and fit the data in the 'clean_text' column of `campaign` to it to create the sparse tf-idf matrix.



In [ ]:
# initialize TfidfVectorizer
...

# Fit vectorizer
X = ...


Next, do the LSA by created a `TruncatedSVD` object (set `n_components=2`) and fit the sparse tf-idf matrix to it using `fit_transform` (see [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)). You should get a matrix with two values for every text document.

In [ ]:
# initialize TruncatedSVD
...

# fit model
Y_lsa = ...

Check the explained variance ratio in the cell below. The ratios are very different from those found in PCA above. Why? (Hint: think about what kind and how many features were originally in the data given to the PCA model versus the LSA model)

In [ ]:
# get the explained variance ratios
...

Now, run the next cell to plot the transformed data.

In [ ]:
# put components in a data frame
lsa_components = pd.DataFrame({'component1': Y_lsa[:, 0],
                          'component2': Y_lsa[:, 1],
                          'Type': y})
# plot
sns.lmplot(x = 'component1', y = 'component2', data = lsa_components, hue='Type', legend=True, fit_reg=False, 
           hue_order=['press release', 'speech', 'statement'], palette=colors, scatter_kws={'s':5}, size=7);

And, take a look at the exact same data, but with the document type marked in colors rather than the candidates.

In [ ]:
# put components in a data frame
lsa_components = pd.DataFrame({'component1': Y_lsa[:, 0],
                          'component2': Y_lsa[:, 1],
                          'Type': campaign.Type})
# plot
sns.lmplot(x = 'component1', y = 'component2', data = lsa_components, hue='Type', legend=True, fit_reg=False, 
            scatter_kws={'s':8}, size=12);

**QUESTION**: How does this plot compare to the one using PCA and extracted numerical features? What does it tell you about the variance of the data? Keep in mind that in this case, the data was fed into LSA as a 'bag of words' for every document- there was no input that explicitly gave the document's type or candidate.

**ANSWER**:

Two important notes on PCA:
1. PCA can also be used while training a model for prediction or classification tasks. In that case, like during any model training, you would set aside your test data before doing PCA to avoid overfitting your model.
2. The components output from PCA can't really be described as any specific input features like 'character length'. They're a set of linearly uncorrelated vectors; we can't break it down much more than than, unfortunately.

### Just for fun: 3 dimensional graphing
Can we see anything new if we ask for 3 principal components instead of 2? Run the following cells to see. Does adding an extra dimension capture anything new about the data?

In [ ]:
# Initialize models
lsa3 = TruncatedSVD(n_components=3)

# Fit models
Y_lsa3 = lsa3.fit_transform(X)

In [ ]:
# Create a new dataframe with the components and some info about each data point
lsa_3components = pd.DataFrame({'component1': Y_lsa3[:, 0],
                          'component2': Y_lsa3[:, 1],
                            'component3': Y_lsa3[:, 2],
                          'Type': campaign.Type,
                          'Candidate': campaign.Candidate, 
                            'Date': campaign.Date})

lsa3.explained_variance_ratio_

Run the cell below to show a 3-dimensional graph of the transformed data points. Try zooming in and out or hiding certain candidates data by clicking on their name on the list at the right side. Do you notice any party or candidate patterns?

In [ ]:
data_comp = []

# plots the texts for each candidate, labeling each point with the date and candidate's color and name
for cand in candidate_colors.keys():
    points = lsa_3components[lsa_3components.Candidate==cand]
    data_comp.append(go.Scatter3d(
        x=points.component1,
        y=points.component2,
        z=points.component3,
        mode='markers',
        marker=dict(size=3,
                line=dict(width=1),
                color=candidate_colors[cand]
               ),
        name=cand, 
        text=points.Date))
    
layout_comp = go.Layout(
    title='LSA',
    hovermode='closest',

)
fig_comp = go.Figure(data=data_comp, layout=layout_comp)
py.iplot(fig_comp, filename='LSA')

---

## Bibliography

- Tukey, J. W. (1977). Exploratory data analysis
- Behrens, J. T. (1997). Principles and procedures of exploratory data analysis. Psychological Methods, 2(2), 131.

---
Notebook developed by: Keeley Takimoto

Data Science Modules: http://data.berkeley.edu/education/modules
